In [49]:
import pyart
import numpy.ma as ma
import numpy as np
from scipy.stats import mstats
from statistics import mean
import warnings
warnings.filterwarnings('ignore')

In [50]:
file_name="PHA210802204004.RAW2RBE"
radar = pyart.io.read_sigmet(file_name)
lat_0 = radar.latitude['data'][0]
lon_0 = radar.longitude['data'][0]
shape_grid = (7, 201,201)

In [51]:
class UF:
    def __init__(self,radar,shape_grid,lat_0, lon_0):
        self.radar = radar
        self.shape = shape_grid
        self.lat_0 = lat_0
        self.lon_0 = lon_0
    def noise_rejection(self):
        #reject noise radar
        range_bins=self.radar.range['data']
        ref_last_bins=self.radar.fields['reflectivity']['data'][:,-5:]
        Z_f=mstats.mode(ref_last_bins,axis=None)[0][0]
        Z_s=pyart.retrieve.compute_noisedBZ(1800, noisedBZ_val=Z_f, _range=range_bins, ref_dist=300., noise_field='noise_dbz')
        Z_splus4=Z_s['data']+4
        conditions_masked=ma.masked_less(self.radar.fields['reflectivity']['data'],Z_splus4)
        #mask Z<0
        self.radar.add_field_like('reflectivity', 'reflectivity_cor_noise',ma.masked_less_equal(conditions_masked,0.),replace_existing=True) 
        #self.radar.add_field_like('reflectivity', 'reflectivity_cor_noise',conditions_masked,replace_existing=True)
    def convert_grid(self):
        grid = pyart.map.grid_from_radars(
            self.radar,
            grid_shape=self.shape, #Number of points in the grid (z, y, x)
            grid_limits=((0, 9000), (-200000, 200000), (-200000, 200000)), # min-max tuong duong z,y,x
            grid_origin = (self.lat_0, self.lon_0),
            fields=['reflectivity_cor_noise'],
            roi_func='dist_beam',
            weighting_function='cressman')
        return grid
    def convert_lat_lon(self, grid):
        #conver distance to lat/lon
        for i in range(self.shape[1]):
            geog = pyart.core.cartesian_to_geographic_aeqd(grid.x["data"][i],grid.y["data"][i],self.lon_0, self.lat_0, R=6370997.0)
            grid.x["data"][i] = geog[0]
            grid.y["data"][i] = geog[1]
        return grid

In [52]:
UF_File = UF(radar, shape_grid, lat_0, lon_0)
ref_cor_noise=UF_File.noise_rejection()
grid = UF_File.convert_grid()
grid_lat_lon = UF_File.convert_lat_lon(grid)

# SHY95

# Short test

In [76]:
YH95=np.zeros((5,5))

In [77]:
lon=grid.x['data'][98:103]
lat=grid.y['data'][98:103]
lons,lats=np.meshgrid(lon,lat)

In [78]:
Zh=np.array([[5, 13, 3, 24, 4],
            [3, 3, 24, 4, 4],
            [5, 3, 30, 3, 13],
            [23, 3, 24, 10, 4],
            [5, 3, 4, 3, 3]])

In [79]:
#Step 1
mask_st1=(Zh>=30)
YH95[mask_st1]=1
print(YH95)
print(mask_st1)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[False False False False False]
 [False False False False False]
 [False False  True False False]
 [False False False False False]
 [False False False False False]]


In [80]:
#Step 2 & 3
def MBG(xo,yo,n,r,Zh): #Mean Background Reflectivity
    y,x = np.ogrid[-xo:n-xo, -yo:n-yo]
    mask = x*x + y*y <= r*r
    Zh_none_O=Zh[xo,yo]
    Zbg = np.ma.masked_array(Zh[mask], Zh[mask] ==Zh_none_O )
    return Zbg

def MBG_mask_r(xo,yo,n,r): # mask array with r change
    y,x = np.ogrid[-xo:n-xo, -yo:n-yo]
    mask = x*x + y*y <= r*r
    return mask

for xo in range (0,5,1):
    for yo in range (0,5,1):
        if YH95[xo,yo]==1:
            Zbg=MBG(xo,yo,5,1.5,Zh).mean()
            if Zbg<10:
                print(Zbg)
                mask_st2=MBG_mask_r(xo,yo,5,r=1)
                YH95[mask_st2]=2
print(YH95)                

9.25
[[0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0.]
 [0. 2. 2. 2. 0.]
 [0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [81]:
print(YH95)

[[0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0.]
 [0. 2. 2. 2. 0.]
 [0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [11]:
for q in range (0,5,1):
    for l in range (0,5,1):
        if Zh[q,l]>0 and YH95[q,l] !=2:
            YH95[q,l]=1

In [12]:
YH95

array([[1., 1., 1., 1., 1.],
       [1., 1., 2., 1., 1.],
       [1., 2., 2., 2., 1.],
       [2., 1., 2., 1., 2.],
       [2., 2., 1., 2., 2.]])